# Vertex LLM Demo: Simplify Complex Information

This demo aims to distill critical points from text and informaton that is more consumable for non-experts to understand.  This could make information delivery more efficient more both the giver and receiver of information, for example a medical provider and a patient.

# CUJs
  * As a patient, I want to understand what this medical note means to me
  * As a provider, I want to focus on critical and objective medical diagnoses and prognoses, and expedite delivery of information to patients.
  * As a product manager, I want to understand the implications of a new technology to my product or future products.
  * and many, many more...

# What the code does

  * The code takes a text as input, such as a medical chart note or complex technical explanation.
  * It then pulls that text into a prompt, instructing the LLM to generate a  summary of the text for an individual at a more appropriate knowledge level (e.g. non-expert, patient, student).
  * The model responds with a summary, and sometimes deeper explanations if certain terms within the summary still require expounding upon.

# Instructions

To run the demo, simply run all the cells in order. The hidden cells can remain hidden and you can run the entire section at once. Note that the workspace setup will trigger a runtime restart; that is working as intended to ensure the right version of the Cloud SDK is installed.


# Limitations and ideas for extension

  * Could possibly be used in a more real-time scenario as part of a chatbot deployed by medical providers / teachers / product teams / etc..
  * The body of text currently needs to fit within the LLM's context window. We can naively solve that problem by chunking up the original body of text and generating summaries of smaller chunks.
  * Some of the input could be sensitive and directly impactful, e.g. in the case of medical information, so the summary would need to be validated prior to a patient, student, individual seeing to ensure minimal or no misinformation.

# Sample output

**Original text**

*72 year old man with h/o CHF following MI, chronic renal insufficiency and venous stasis admitted with worsening edema and DOE. His symptoms are most consistent with incrasing CHF-biventricular-which would account for both his pulmonary congestion as well as his peripheral edema. His renal disease is a less likely explanation for his extensive edema as his BUN/Cr have remained stable throughout. However, his low albumin which could contribute to his edema may be due to renal losses. So if his edema is due to CHF, why has it become gradually and now acutely worse? Possibilities include: 1) worsening LV function, 2) another MI, 3) worsening valvular disease, 4) poor compliance with medications or 5) excess salt and water intake. His ECHO today shows no change in his EF, but there is marked wall motion abnormalities with akinesis. There is no evidence in his history, EKG, or enzymes for current ischemia/infarct. He does have MR and TR and his valvular disease may in part account for his worsening symptoms though his estimated PA pressure is unchanged and his LA is not dilated. The most likely precipitant of his failure is a combination of poor compliance with medication and fluid overload from excessive intake. We will continue to investigate the possibility of a structural precipitant for his deterioration and treat his current symptoms.*

**Response - non-expert summary (text-bison-001 model)**

*72 year old man with heart failure due to previous heart attack, kidney disease and poor circulation is admitted to hospital with worsening symptoms. Doctors are trying to figure out what caused his condition to get worse. They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt. They will do more tests to find out the cause and treat his symptoms.*

**Expanded Response - explanation of non-expert summary (text-bison-001 model)**

*72 year old man with heart failure due to previous heart attack, kidney disease and poor circulation is admitted to hospital with worsening symptoms. Doctors are trying to figure out what caused his condition to get worse. They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt. They will do more tests to find out the cause and treat his symptoms.*

*A 72-year-old man with heart failure due to a previous heart attack, kidney disease, and poor circulation was admitted to the hospital with worsening symptoms. The doctors are trying to figure out what caused his condition to get worse. They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt. They will do more tests to find out the cause and treat his symptoms.*

*Heart failure is a condition in which the heart cannot pump enough blood to meet the body's needs. This can lead to symptoms such as shortness of breath, fatigue, and swelling in the feet, ankles, and legs. Heart failure can be caused by a number of factors, including heart attack, high blood pressure, and diabetes.*

*Kidney disease is a condition in which the kidneys cannot function properly. This can lead to symptoms such as swelling, fatigue, and urination problems. Kidney disease can be caused by a number of factors, including high blood pressure, diabetes, and smoking.*

*Poor circulation is a condition in which the blood does not flow properly through the body. This can lead to symptoms such as coldness, numbness, and pain in the feet, ankles, and legs. Poor circulation can be caused by a number of factors, including high blood pressure, diabetes, and smoking.*

*The doctors are trying to figure out what caused the man's condition to get worse. They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt. They will do more tests to find out the cause and treat his symptoms.*

*The man's treatment will depend on the cause of his condition. If his heart is not working as well as it used to, he may need surgery to repair or replace his heart. If he has another heart problem, he may need medication to treat it. If he is not taking his medication properly, he will need to be started on a new medication regimen. If he is drinking too much water and eating too much salt, he will need to be put on a diet that restricts his fluid and salt intake.*

*The man's prognosis will depend on the cause of his condition and how well he responds to treatment. If his heart is not working as well as it used to, he may have a poor prognosis. If he has another heart problem, he may also have a poor prognosis. However, if he is taking his medication properly and is not drinking too much water and eating too much salt, he may have a good prognosis.*



## LLM Setup ##

---

This section sets up an easy interface for interaction with Vertex LLM.

---

In [1]:
from datetime import datetime
import time
import pandas as pd
import numpy as np
import json
import os
import matplotlib.pyplot as plt

from IPython.display import Markdown

import vertexai
from vertexai.language_models import TextGenerationModel, TextEmbeddingModel
from vertexai.preview.language_models import TextGenerationModel as TextGenerationModel_preview
from google.cloud import aiplatform
from google.cloud import storage
from google.cloud import bigquery

print("Vertex AI version: " + str(aiplatform.__version__))

2023-09-20 09:31:27.357506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Vertex AI version: 1.32.0


### Env variables and Setup

In [5]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [6]:
REGION = 'us-central1'
LOCATION = 'us'

In [7]:
VERTEX_LLM = 'text-bison@latest'
VERTEX_LLM_32k = 'text-bison-32k'

In [8]:
# gcs client
gcs = storage.Client(project = PROJECT_ID)

# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)
aiplatform.init(project = PROJECT_ID, location = REGION)

## Upload Sample Body of Text##

---

This section provides some example medical chart notes that a patient may see when provided by a provider.

---



In [12]:
# pulled from sources such as: https://www.mtsamples.com/site/pages/browse.asp?Type=87-Office%20Notes

text_samples_med = [
"""
72 year old man with h/o CHF following MI, chronic renal insufficiency and venous stasis admitted with worsening edema and DOE.
His symptoms are most consistent with incrasing CHF-biventricular-which would account for both his pulmonary congestion as well as his peripheral edema.
His renal disease is a less likely explanation for his extensive edema as his BUN/Cr have remained stable throughout.
However, his low albumin which could contribute to his edema may be due to renal losses.
So if his edema is due to CHF, why has it become gradually and now acutely worse? Possibilities include:
1) worsening LV function, 2) another MI, 3) worsening valvular disease, 4) poor compliance with medications or
5) excess salt and water intake. His ECHO today shows no change in his EF, but there is marked wall motion abnormalities with akinesis.
There is no evidence in his history, EKG, or enzymes for current ischemia/infarct.
He does have MR and TR and his valvular disease may in part account for his worsening symptoms though his estimated PA pressure is unchanged and his LA is not dilated.
The most likely precipitant of his failure is a combination of poor compliance with medication and fluid overload from excessive intake.
We will continue to investigate the possibility of a structural precipitant for his deterioration and treat his current symptoms.
"""
,"""
This 66-year-old white male was seen in my office on Month DD, YYYY. Patient was recently discharged from Doctors Hospital at Parkway after he was treated for pneumonia.
Patient continues to have severe orthopnea, paroxysmal nocturnal dyspnea, cough with greenish expectoration.
His exercise tolerance is about two to three yards for shortness of breath.
The patient stopped taking Coumadin for reasons not very clear to him. He was documented to have recent atrial fibrillation.
Patient has longstanding history of ischemic heart disease, end-stage LV systolic dysfunction, and is status post ICD implantation.
Fasting blood sugar this morning is 130.
"""
,"""
This 32-year-old with family history of premature coronary artery disease came in for evaluation of recurrent chest pain, O2 saturation at 94% with both atypical and typical features of ischemia.
The patient ruled out for myocardial infarction with serial troponins.
Nuclear stress test has been done, results of which are pending.
The patient is stable to be discharged pending the results of nuclear stress test and cardiologist's recommendations.
He will follow up with cardiologist, Dr. X, in two weeks and with his primary physician in two to four weeks.
Discharge medications will depend on results of nuclear stress test.
"""
,"""
The patient returns to our office today because of continued problems with her headaches.
She was started on Zonegran on her last visit and she states that initially she titrated upto 100 mg q.h.s.
Initially felt that the Zonegran helped, but then the pain in her head returned.
It is an area of tenderness and sensitivity in her left parietal area.
It is a very localized pain.
She takes Motrin 400 mg b.i.d., which helped.
She also had EMG/nerve conduction studies since she was last seen in our office that showed severe left ulnar neuropathy, moderate right ulnar neuropathy, bilateral mild-to-moderate carpal tunnel and diabetic neuropathy.
She was referred to Dr. XYZ and will be seeing him on August 8, 2006.
She was also never referred to the endocrine clinic to deal with her poor diabetes control.
Her last hemoglobin A1c was 10.
"""
]
text_samples_tech_math = [
"""
In probability theory and statistics, Bayes' theorem (alternatively Bayes' law or Bayes' rule), named after Thomas Bayes, describes the probability of an event, based on prior knowledge of conditions that might be related to the event.For example, if the risk of developing health problems is known to increase with age, Bayes' theorem allows the risk to an individual of a known age to be assessed more accurately by conditioning it relative to their age, rather than simply assuming that the individual is typical of the population as a whole.
One of the many applications of Bayes' theorem is Bayesian inference, a particular approach to statistical inference. When applied, the probabilities involved in the theorem may have different probability interpretations. With Bayesian probability interpretation, the theorem expresses how a degree of belief, expressed as a probability, should rationally change to account for the availability of related evidence. Bayesian inference is fundamental to Bayesian statistics, being considered by one authority as; "to the theory of probability what Pythagoras's theorem is to geometry.
"""
,"""
Compound interest is the addition of interest to the principal sum of a loan or deposit, or in other words, interest on principal plus interest. It is the result of reinvesting interest, or adding it to the loaned capital rather than paying it out, or requiring payment from borrower, so that interest in the next period is then earned on the principal sum plus previously accumulated interest. Compound interest is standard in finance and economics.
Compound interest is contrasted with simple interest, where previously accumulated interest is not added to the principal amount of the current period, so there is no compounding. The simple annual interest rate is the interest amount per period, multiplied by the number of periods per year. The simple annual interest rate is also known as the nominal interest rate (not to be confused with the interest rate not adjusted for inflation, which goes by the same name).
"""
,"""
An application programming interface (API) is a way for two or more computer programs to communicate with each other. It is a type of software interface, offering a service to other pieces of software.[1] A document or standard that describes how to build or use such a connection or interface is called an API specification. A computer system that meets this standard is said to implement or expose an API. The term API may refer either to the specification or to the implementation.
In contrast to a user interface, which connects a computer to a person, an application programming interface connects computers or pieces of software to each other. It is not intended to be used directly by a person (the end user) other than a computer programmer who is incorporating it into the software. An API is often made up of different parts which act as tools or services that are available to the programmer. A program or a programmer that uses one of these parts is said to call that portion of the API. The calls that make up the API are also known as subroutines, methods, requests, or endpoints. An API specification defines these calls, meaning that it explains how to use or implement them.
One purpose of APIs is to hide the internal details of how a system works, exposing only those parts a programmer will find useful and keeping them consistent even if the internal details later change. An API may be custom-built for a particular pair of systems, or it may be a shared standard allowing interoperability among many systems.
There are APIs for programming languages, software libraries, computer operating systems, and computer hardware. APIs originated in the 1940s, though the term did not emerge until the 1960s and 1970s. Contemporary usage of the term API often refers to web APIs,[2] which allow communication between computers that are joined by the internet. Recent developments in APIs have led to the rise in popularity of microservices, which are loosely coupled services accessed through public APIs.
"""
]

In [13]:
print(text_samples_med[0])
print(text_samples_tech_math[0])


72 year old man with h/o CHF following MI, chronic renal insufficiency and venous stasis admitted with worsening edema and DOE.
His symptoms are most consistent with incrasing CHF-biventricular-which would account for both his pulmonary congestion as well as his peripheral edema.
His renal disease is a less likely explanation for his extensive edema as his BUN/Cr have remained stable throughout.
However, his low albumin which could contribute to his edema may be due to renal losses.
So if his edema is due to CHF, why has it become gradually and now acutely worse? Possibilities include:
1) worsening LV function, 2) another MI, 3) worsening valvular disease, 4) poor compliance with medications or
5) excess salt and water intake. His ECHO today shows no change in his EF, but there is marked wall motion abnormalities with akinesis.
There is no evidence in his history, EKG, or enzymes for current ischemia/infarct.
He does have MR and TR and his valvular disease may in part account for his 

## Generate Summaries ##

---

Here we create a basic summary generation prompt and print the results.

Results are OK.

---

In [14]:
textgen_model_32k = TextGenerationModel_preview.from_pretrained(VERTEX_LLM_32k)
textgen_model = TextGenerationModel.from_pretrained(VERTEX_LLM)

In [15]:
# Ask the LLM
for text in text_samples_med:
    prompt = """
    Summarize the key points of the text below:

    Text:
    {}
    """.format(text)

    # Send prompt to LLM
    response = textgen_model.predict(
       (prompt),
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    )
    display(Markdown("## Original text"))
    display(Markdown(text))
    display(Markdown("### Response"))
    display(Markdown(str(response)))
    print('-------------------------')

## Original text


72 year old man with h/o CHF following MI, chronic renal insufficiency and venous stasis admitted with worsening edema and DOE.
His symptoms are most consistent with incrasing CHF-biventricular-which would account for both his pulmonary congestion as well as his peripheral edema.
His renal disease is a less likely explanation for his extensive edema as his BUN/Cr have remained stable throughout.
However, his low albumin which could contribute to his edema may be due to renal losses.
So if his edema is due to CHF, why has it become gradually and now acutely worse? Possibilities include:
1) worsening LV function, 2) another MI, 3) worsening valvular disease, 4) poor compliance with medications or
5) excess salt and water intake. His ECHO today shows no change in his EF, but there is marked wall motion abnormalities with akinesis.
There is no evidence in his history, EKG, or enzymes for current ischemia/infarct.
He does have MR and TR and his valvular disease may in part account for his worsening symptoms though his estimated PA pressure is unchanged and his LA is not dilated.
The most likely precipitant of his failure is a combination of poor compliance with medication and fluid overload from excessive intake.
We will continue to investigate the possibility of a structural precipitant for his deterioration and treat his current symptoms.


### Response

 72-year-old man with a history of CHF, chronic renal insufficiency, and venous stasis is admitted with worsening edema and DOE.
His symptoms are most consistent with increasing CHF-biventricular.
His renal disease is a less likely explanation for his extensive edema.
His low albumin may be due to renal losses.
His ECHO today shows no change in his EF, but there is marked wall motion abnormalities with akinesis.
The most likely precipitant of his failure is a combination of poor compliance with medication and fluid overload from excessive intake.

-------------------------


...

## Original text


The patient returns to our office today because of continued problems with her headaches.
She was started on Zonegran on her last visit and she states that initially she titrated upto 100 mg q.h.s.
Initially felt that the Zonegran helped, but then the pain in her head returned.
It is an area of tenderness and sensitivity in her left parietal area.
It is a very localized pain.
She takes Motrin 400 mg b.i.d., which helped.
She also had EMG/nerve conduction studies since she was last seen in our office that showed severe left ulnar neuropathy, moderate right ulnar neuropathy, bilateral mild-to-moderate carpal tunnel and diabetic neuropathy.
She was referred to Dr. XYZ and will be seeing him on August 8, 2006.
She was also never referred to the endocrine clinic to deal with her poor diabetes control.
Her last hemoglobin A1c was 10.


### Response

 The patient has been experiencing headaches and was prescribed Zonegran, which initially helped but then the pain returned.
She has tenderness and sensitivity in her left parietal area and takes Motrin 400 mg b.i.d. for the pain.
EMG/nerve conduction studies showed severe left ulnar neuropathy, moderate right ulnar neuropathy, bilateral mild-to-moderate carpal tunnel and diabetic neuropathy.
She has been referred to Dr. XYZ for further evaluation and will be seeing him on August 8, 2006.
She has not been referred to the endocrine clinic to deal with her poor diabetes control and her last hemoglobin A1c was 10.

-------------------------


In [17]:
# Ask the LLM
for text in text_samples_tech_math:
    prompt = """
    Summarize the key points of the text below:

    Text:
    {}
    """.format(text)

    # Send prompt to LLM
    response = textgen_model.predict(
       (prompt),
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    )
    display(Markdown("## Original text"))
    display(Markdown(text))
    display(Markdown("### Response"))
    display(Markdown(str(response)))
    print('-------------------------')

## Original text


In probability theory and statistics, Bayes' theorem (alternatively Bayes' law or Bayes' rule), named after Thomas Bayes, describes the probability of an event, based on prior knowledge of conditions that might be related to the event.For example, if the risk of developing health problems is known to increase with age, Bayes' theorem allows the risk to an individual of a known age to be assessed more accurately by conditioning it relative to their age, rather than simply assuming that the individual is typical of the population as a whole.
One of the many applications of Bayes' theorem is Bayesian inference, a particular approach to statistical inference. When applied, the probabilities involved in the theorem may have different probability interpretations. With Bayesian probability interpretation, the theorem expresses how a degree of belief, expressed as a probability, should rationally change to account for the availability of related evidence. Bayesian inference is fundamental to Bayesian statistics, being considered by one authority as; "to the theory of probability what Pythagoras's theorem is to geometry.


### Response

 Bayes' theorem is a fundamental theorem in probability theory and statistics that describes the probability of an event based on prior knowledge of conditions that might be related to the event. It allows for the assessment of the risk to an individual of a known age, for example, by conditioning it relative to their age, rather than simply assuming that the individual is typical of the population as a whole. One of the many applications of Bayes' theorem is Bayesian inference, a particular approach to statistical inference. When applied, the probabilities involved in the theorem may have different probability interpretations. With Bayesian probability interpretation, the theorem expresses how a degree of belief, expressed as a probability, should rationally change to account for the availability of related evidence. Bayesian inference is fundamental to Bayesian statistics.

-------------------------


...

## Original text


An application programming interface (API) is a way for two or more computer programs to communicate with each other. It is a type of software interface, offering a service to other pieces of software.[1] A document or standard that describes how to build or use such a connection or interface is called an API specification. A computer system that meets this standard is said to implement or expose an API. The term API may refer either to the specification or to the implementation.
In contrast to a user interface, which connects a computer to a person, an application programming interface connects computers or pieces of software to each other. It is not intended to be used directly by a person (the end user) other than a computer programmer who is incorporating it into the software. An API is often made up of different parts which act as tools or services that are available to the programmer. A program or a programmer that uses one of these parts is said to call that portion of the API. The calls that make up the API are also known as subroutines, methods, requests, or endpoints. An API specification defines these calls, meaning that it explains how to use or implement them.
One purpose of APIs is to hide the internal details of how a system works, exposing only those parts a programmer will find useful and keeping them consistent even if the internal details later change. An API may be custom-built for a particular pair of systems, or it may be a shared standard allowing interoperability among many systems.
There are APIs for programming languages, software libraries, computer operating systems, and computer hardware. APIs originated in the 1940s, though the term did not emerge until the 1960s and 1970s. Contemporary usage of the term API often refers to web APIs,[2] which allow communication between computers that are joined by the internet. Recent developments in APIs have led to the rise in popularity of microservices, which are loosely coupled services accessed through public APIs.


### Response

 - An API (Application Programming Interface) is a set of protocols that allows different software components to communicate with each other.


- APIs are not intended to be used directly by end-users, but rather by programmers who incorporate them into their own software.


- APIs can be custom-built for specific systems or standardized to allow interoperability among many systems.


- APIs exist for programming languages, software libraries, operating systems, and hardware.


- APIs originated in the 1940s, but the term was coined in the 1960s and 1970s.


- Today, APIs are often associated with web APIs that enable communication between computers over the internet.


- The rise of microservices, which are loosely coupled services accessed through public APIs, has contributed to the popularity of APIs.

-------------------------



## Generate Non-Expert Summaries ##

---

Here we create a summary generation prompt that aims to simplify the content for non-experts and print the results.

Results are better than the basic prompt.

---

In [18]:
summaries = []
summaries

[]

In [19]:
# Ask the LLM
for text in text_samples_med:
    prompt = """
    Generate a summary that a non-expert would understand of the following text:

    Text:
    {}
    """.format(text)

    # Send prompt to LLM
    response = textgen_model.predict(
       (prompt),
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    )
    summaries.append(response)
    display(Markdown("## Original text"))
    display(Markdown(text))
    display(Markdown("### Response"))
    display(Markdown(str(response)))
    print('-------------------------')

## Original text


72 year old man with h/o CHF following MI, chronic renal insufficiency and venous stasis admitted with worsening edema and DOE.
His symptoms are most consistent with incrasing CHF-biventricular-which would account for both his pulmonary congestion as well as his peripheral edema.
His renal disease is a less likely explanation for his extensive edema as his BUN/Cr have remained stable throughout.
However, his low albumin which could contribute to his edema may be due to renal losses.
So if his edema is due to CHF, why has it become gradually and now acutely worse? Possibilities include:
1) worsening LV function, 2) another MI, 3) worsening valvular disease, 4) poor compliance with medications or
5) excess salt and water intake. His ECHO today shows no change in his EF, but there is marked wall motion abnormalities with akinesis.
There is no evidence in his history, EKG, or enzymes for current ischemia/infarct.
He does have MR and TR and his valvular disease may in part account for his worsening symptoms though his estimated PA pressure is unchanged and his LA is not dilated.
The most likely precipitant of his failure is a combination of poor compliance with medication and fluid overload from excessive intake.
We will continue to investigate the possibility of a structural precipitant for his deterioration and treat his current symptoms.


### Response

 A 72-year-old man with a history of congestive heart failure (CHF), chronic kidney disease, and venous stasis is admitted to the hospital with worsening edema (swelling) and difficulty breathing.

His symptoms are most likely due to worsening CHF. CHF can cause both pulmonary congestion (fluid in the lungs) and peripheral edema (swelling in the feet and ankles).

His renal disease is a less likely explanation for his extensive edema because his kidney function has remained stable.

However, his low albumin level, which can contribute to edema, may be due to renal losses.

There are several possible reasons why his CHF has become gradually and now acutely worse. These include:
* Worsening left ventricular function
* Another heart attack
* Worsening valvular disease
* Poor compliance with medications
* Excess salt and water intake

His echocardiogram (ECHO) today shows no change in his ejection fraction (EF), but there are marked wall motion abnormalities with akinesis (absence of movement).

There is no evidence in his history, EKG, or enzymes for current ischemia/infarct (heart attack).

He does have mitral regurgitation (MR) and tricuspid regurgitation (TR), and his valvular disease may in part account for his worsening symptoms. However, his estimated pulmonary artery pressure is unchanged and his left atrium is not dilated.

The most likely precipitant of his failure is a combination of poor compliance with medication and fluid overload from excessive intake.

The patient will continue to be investigated for a structural precipitant for his deterioration and treated for his current symptoms.

-------------------------


...

## Original text


The patient returns to our office today because of continued problems with her headaches.
She was started on Zonegran on her last visit and she states that initially she titrated upto 100 mg q.h.s.
Initially felt that the Zonegran helped, but then the pain in her head returned.
It is an area of tenderness and sensitivity in her left parietal area.
It is a very localized pain.
She takes Motrin 400 mg b.i.d., which helped.
She also had EMG/nerve conduction studies since she was last seen in our office that showed severe left ulnar neuropathy, moderate right ulnar neuropathy, bilateral mild-to-moderate carpal tunnel and diabetic neuropathy.
She was referred to Dr. XYZ and will be seeing him on August 8, 2006.
She was also never referred to the endocrine clinic to deal with her poor diabetes control.
Her last hemoglobin A1c was 10.


### Response

 The patient is returning to the office because her headaches have not improved. 
She was prescribed Zonegran, but it did not help. 
She has pain in her left parietal area that is relieved by Motrin. 
She also has diabetic neuropathy and carpal tunnel syndrome. 
She has been referred to a neurologist and an endocrinologist.

-------------------------


In [20]:
# Ask the LLM
for text in text_samples_tech_math:
    prompt = """
    Generate a summary that a non-expert would understand of the following text:

    Text:
    {}
    """.format(text)

    # Send prompt to LLM
    response = textgen_model.predict(
       (prompt),
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    )
    summaries.append(response)
    display(Markdown("## Original text"))
    display(Markdown(text))
    display(Markdown("### Response"))
    display(Markdown(str(response)))
    print('-------------------------')

## Original text


In probability theory and statistics, Bayes' theorem (alternatively Bayes' law or Bayes' rule), named after Thomas Bayes, describes the probability of an event, based on prior knowledge of conditions that might be related to the event.For example, if the risk of developing health problems is known to increase with age, Bayes' theorem allows the risk to an individual of a known age to be assessed more accurately by conditioning it relative to their age, rather than simply assuming that the individual is typical of the population as a whole.
One of the many applications of Bayes' theorem is Bayesian inference, a particular approach to statistical inference. When applied, the probabilities involved in the theorem may have different probability interpretations. With Bayesian probability interpretation, the theorem expresses how a degree of belief, expressed as a probability, should rationally change to account for the availability of related evidence. Bayesian inference is fundamental to Bayesian statistics, being considered by one authority as; "to the theory of probability what Pythagoras's theorem is to geometry.


### Response

 Bayes' theorem is a way of calculating the probability of an event based on prior knowledge of conditions that might be related to the event. For example, if we know that the risk of developing health problems increases with age, we can use Bayes' theorem to calculate the risk to an individual of a known age more accurately than if we simply assumed that the individual is typical of the population as a whole. 

One of the many applications of Bayes' theorem is Bayesian inference, which is a particular approach to statistical inference. When applied, the probabilities involved in the theorem may have different probability interpretations. With Bayesian probability interpretation, the theorem expresses how a degree of belief, expressed as a probability, should rationally change to account for the availability of related evidence. Bayesian inference is fundamental to Bayesian statistics.

-------------------------


...

## Original text


An application programming interface (API) is a way for two or more computer programs to communicate with each other. It is a type of software interface, offering a service to other pieces of software.[1] A document or standard that describes how to build or use such a connection or interface is called an API specification. A computer system that meets this standard is said to implement or expose an API. The term API may refer either to the specification or to the implementation.
In contrast to a user interface, which connects a computer to a person, an application programming interface connects computers or pieces of software to each other. It is not intended to be used directly by a person (the end user) other than a computer programmer who is incorporating it into the software. An API is often made up of different parts which act as tools or services that are available to the programmer. A program or a programmer that uses one of these parts is said to call that portion of the API. The calls that make up the API are also known as subroutines, methods, requests, or endpoints. An API specification defines these calls, meaning that it explains how to use or implement them.
One purpose of APIs is to hide the internal details of how a system works, exposing only those parts a programmer will find useful and keeping them consistent even if the internal details later change. An API may be custom-built for a particular pair of systems, or it may be a shared standard allowing interoperability among many systems.
There are APIs for programming languages, software libraries, computer operating systems, and computer hardware. APIs originated in the 1940s, though the term did not emerge until the 1960s and 1970s. Contemporary usage of the term API often refers to web APIs,[2] which allow communication between computers that are joined by the internet. Recent developments in APIs have led to the rise in popularity of microservices, which are loosely coupled services accessed through public APIs.


### Response

 An API is a set of instructions that allows different software components to communicate with each other. It's like a waiter taking your order at a restaurant - the waiter doesn't need to know how the food is cooked, they just need to know what you want and how to get it to you. APIs are used to connect different parts of a software system, or to connect different software systems together. They can be used to exchange data, perform actions, or get information. APIs are essential for building complex software systems, and they're becoming increasingly important as more and more devices and services are connected to the internet.

-------------------------



## Generate Explanations Based on Simplified Summaries ##

---

Here we create an explanation generation prompt that aims to expand upon the summaries for non-experts and print the results.

Results are ___.

---

In [21]:
len(summaries)

7

In [ ]:
for summary in summaries:

    generation_prompt = """
    Generate a more detailed explanation that a non-expert would understand of the following text.

    Previous Summary:
    {}

    Explanation:
    """.format(summary)

    # Send prompt to LLM
    response = textgen_model.predict(
       (prompt),
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    )
    summaries.append(response)
    display(Markdown("## Original text"))
    display(Markdown(summary))
    display(Markdown("### Response"))
    display(Markdown(str(response)))
    print('-------------------------')



## Generate Even Simpler Summaries ##

---

Here we create a summary generation prompt that aims to simplify the content to even a lower level and use bullets to format and print the results.

Results are generally similar to the previous attempt, and don't always include bullet points in the response.

---

In [ ]:
for text in text_samples_med:

    generation_prompt = """
    Generate a 3-5 bullet-point summary that a 6th grader would understand of the following text.

    Text:
    {}

    Summary:
    """.format(text)

    # Send prompt to LLM
    response = textgen_model.predict(
       (prompt),
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    )
    summaries.append(response)
    display(Markdown("## Original text"))
    display(Markdown(text))
    display(Markdown("### Response"))
    display(Markdown(str(response)))
    print('-------------------------')


In [ ]:
for text in text_samples_tech_math:

    generation_prompt = """
    Generate a 3-5 bullet-point summary that a 6th grader would understand of the following text.

    Text:
    {}

    Summary:
    """.format(text)

    # Send prompt to LLM
    response = textgen_model.predict(
       (prompt),
        max_output_tokens=1024,
        temperature=0.4,
        top_p=0.8,
        top_k=40,
    )
    summaries.append(response)
    display(Markdown("## Original text"))
    display(Markdown(text))
    display(Markdown("### Response"))
    display(Markdown(str(response)))
    print('-------------------------')
